In [38]:
!pip install -U dspy-ai


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [29]:
import os
import json
import dspy
import re

In [30]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [31]:
from pydantic import BaseModel, Field, field_validator,model_validator
from pydantic import ValidationError
from typing import List

In [35]:
class GameOf24(dspy.Signature):
    """
    Finalize the mathematical expression for the Game of 24.
    
    STRICT RULES:
    1. The expression MUST use all four initial numbers from 'problem' exactly once.
    2. The expression MUST be mathematically equivalent to the target (24) or the final result of 'reasoning_steps'.
    3. Use only the operators and pairings defined in the 'reasoning_steps'.
    4. Output ONLY the expression string.
    """
    problem = dspy.InputField(desc="The initial four numbers")
    reasoning_steps = dspy.InputField(desc="The sequence of valid moves found")
    expression = dspy.OutputField(desc="The final formula using parentheses, e.g., (4 * (5 + 1))")


In [46]:

# Enable DSPy internal logging
dspy.enable_logging()

 
class MathStep(BaseModel):
    """
    A single valid move in the Game of 24.
    STRICT RULES:
    1. 'result' MUST be the exact mathematical outcome of (num1 operator num2).
    2. NEVER use 'null', 'None', or strings like 'Error' in any field.
    3. 'remaining' must be the new list of numbers after removing num1, num2 and adding result to available_numbers.
    """
    num1: int = Field(description="First number from available_numbers") 
    operator: str = Field(description="One of: +, -, *, /") 
    num2: int = Field(description="Second number from available_numbers") 
    # Kept as float to match your original requirement
    result: float = Field(description="The STRICT mathematical result of (num1 operator num2)",gt=float("-inf"))
    remaining: List[int] = Field(description="The list of numbers left after this move")
    
    @model_validator(mode='before')
    @classmethod
    def check_for_nulls(cls, data):
        # If the LLM sent null/None for required fields, raise error immediately
        if isinstance(data, dict):
            required = ['num1', 'operator', 'num2', 'result']
            for field in required:
                if data.get(field) is None:
                    raise ValueError(f"Field '{field}' cannot be null. Propose a valid move.")
        return data
        
    @field_validator('result', mode='before')
    @classmethod
    def coerce_result(cls, v):
        # If the LLM sent an "Error: ..." string, this forces a failure 
        # that DSPy can catch instead of crashing the Python process.
        if isinstance(v, str):
            import re
            # Extract only the first number found in the string
            numbers = re.findall(r"[-+]?\d*\.\d+|\d+", v)
            if not numbers:
                raise ValueError(f"Result must be a number, got: {v}")
            return float(numbers[0])
        return v
        
    @field_validator('result')
    @classmethod
    def validate_math_result(cls, v, info):
        # Safely get values from already-validated fields
        n1 = info.data.get('num1')
        n2 = info.data.get('num2')
        op = info.data.get('operator')
        
        # Ensure all values exist before performing modulo
        if n1 is not None and n2 is not None and op is not None:
            if op == '/':
                if n2 == 0:
                    raise ValueError("Division by zero is illegal.")
                if n1 % n2 != 0:
                    raise ValueError(f"STRICT move illegal: {n1} / {n2} results in a float ({v}). Only integer division allowed.")
        return v
    
  
class ProposeThoughts(dspy.Signature):
    """Propose up to 10 distinct mathematical moves.
    STRICT RULES:
    1. You MUST ONLY use numbers from the 'available_numbers' list.
    2. Do NOT use numbers if they are not in the list. 
    3. 'result' MUST be a number. NEVER use 'null'. 
    4. Format the 'proposals' field as a clean JSON list of MathStep objects, no need to report errors. 
    5. 'remaining' MUST be included: start with available_numbers, remove num1 and num2, then add result.
    6. Do NOT include extra fields like 'title' or error messages.
    8. 'result' MUST be an integer if the operator is division.
    """
    available_numbers: list[int] = dspy.InputField()
    target: int = dspy.InputField()
    proposals: list[MathStep] = dspy.OutputField()

    
# 1. Use Pydantic for structured scoring
class Evaluation(BaseModel):
    score: float = Field(description="A numeric score between 1 and 10")
    reasoning: str = Field(description="Brief explanation of the score")

class EvaluateThought(dspy.Signature):
    """
    Evaluate if a mathematical move brings us closer to the target.
    Score 10: The move hits the target exactly.
    Score 7-9: The move creates a number that is a factor or multiple of the target.
    Score 1: The move creates a large prime or number that makes the target impossible.
    OUTPUT ONLY the raw assessment data. 
    DO NOT include 'Score:' labels or conversational text.
    """
    target: str = dspy.InputField(desc="The number we are trying to reach")
    thought: str = dspy.InputField(desc="The specific math move being scored")
   
    assessment: Evaluation = dspy.OutputField(desc="Score and reasoning") # DSPy handles the parsing automatically

def oldmove(step: MathStep, available_numbers: list[int]) -> bool:
    """
    Validates if a move is legal using the ops dictionary logic.
    Returns True if the move is valid and results in an integer.
    """
    # 1. Standardize inputs to integers for reliable list lookups
    try:
        a = int(step.num1)
        b = int(step.num2)
        claimed_result = float(step.result)
    except (ValueError, TypeError):
        return False

    # 2. Check Number Availability (Hallucination Check)
    temp_nums = list(available_numbers)
    if a not in temp_nums:
        print(f"❌ Hallucination Detected: {a} not in {temp_nums}")
        return False
    temp_nums.remove(a)
    
    if b not in temp_nums:
        print(f"❌ Hallucination Detected: {b} not in {temp_nums}")
        return False

    # 3. Use your 'ops' dictionary logic with the float-illegal check
    ops = {
        '+': lambda x, y: x + y, 
        '-': lambda x, y: x - y, 
        '*': lambda x, y: x * y, 
        '/': lambda x, y: (x / y) if (y != 0 and x % y == 0) else None
    }
    
    calc_res = ops.get(step.operator, lambda x, y: None)(a, b)

    # 4. Strict Validation
    # Fails if result is None (div by zero or float div) 
    # or if the LLM's result doesn't match the math
    if calc_res is None or float(calc_res) != claimed_result:
        print(f"❌ Logic Pass Failed: {a} {step.operator} {b} != {claimed_result}")
        return False

    # 5. Success
    print(f"✅ Logic Pass: {a} {step.operator} {b} = {calc_res}")
    return True
    

def validate_move(step: MathStep, available: list[int]):
    """Ground truth: check if the move is valid and update remaining numbers."""
    nums = available.copy()

    # Remove one occurrence of each operand
    for operand in [int(step.num1), int(step.num2)]:
        try:
            nums.remove(operand)
        except ValueError:
            return False, []  # Operand not available

    # Define operations
    ops = {
        '+': lambda x, y: x + y,
        '-': lambda x, y: x - y,
        '*': lambda x, y: x * y,
        '/': lambda x, y: x / y if y != 0 else None
    }

    calc_res = ops.get(step.operator, lambda x, y: None)(step.num1, step.num2)

    # Reject invalid math or mismatched result
    if calc_res is None or abs(calc_res - step.result) > 0.001:
        return False, []

    # Enforce integer division
    if step.operator == '/' and calc_res != int(calc_res):
        return False, []

    # Append the computed result
    nums.append(int(calc_res))
    return True, sorted(nums)
 

# 2. Build the ToT Module
class TreeOfThoughts(dspy.Module):
    def __init__(self, depth=2):
        super().__init__()
        self.proposer = dspy.ChainOfThought(ProposeThoughts)
        self.evaluator = dspy.ChainOfThought(EvaluateThought) 
        # ADD THIS: The final solver
        self.solver = dspy.Predict(GameOf24) 
        self.depth = depth

    def forward(self, problem,target):
        target_val = float(target)
        # Start with the initial problem state
        # SEED THE STATE: Make it impossible for the model to miss the numbers
        
        # Extract numbers from the problem string
        initial_nums = [int(x.strip()) for x in problem.split(",") if x.strip().isdigit()]
        
        candidates = [{"steps": f"Goal: {target}", "available": initial_nums, "score": 0.0}]

        for d in range(self.depth):
            print(f"depth is:+{d}")
            new_candidates = []
            for candidate in candidates:

                
                try:
                    # Pydantic validation happens INSIDE this call
                    res = self.proposer(
                        available_numbers=candidate["available"],
                        target=target,
                        current_steps=candidate["steps"]
                    )
                except ValidationError as e:
                    print(f"⚠️ Entire proposal block failed validation: {e}")
                    continue # Skip this candidate and try the next one
            

                if not res.proposals: continue
                    
                for raw_move in res.proposals:
                    try:
                        # If already a MathStep, keep it
                        if isinstance(raw_move, MathStep):
                            step = raw_move
                        else:
                            # Validate one move at a time
                            step = MathStep(**raw_move)
                
                        # Ground truth check
                        is_valid, actual_remaining = validate_move(step,candidate["available"])
                        if not is_valid: 
                            print(f"❌ Logic/Hallucination Failed: {step.num1} {step.operator} {step.num2}") 
                            continue
                       
                        # Format the move string for reasoning and scoring
                        move_str = f"{step.num1} {step.operator} {step.num2} = {step.result}"
                        current_path = candidate["steps"] + f" -> {move_str}"

                        #  Success Check
                        if abs(step.result - target_val) < 0.001 and len(actual_remaining) == 1:
                            print(f"🎯 TARGET FOUND: {move_str}")
                            final = self.solver(problem=problem, reasoning_steps=current_path)
                            return dspy.Prediction(answer=final.expression, path=current_path, score=10.0)
                         
                        print(f"✅ Logic Pass: {move_str} (Remaining: {actual_remaining})")
                            
                        #Evaluate and keep searching
                        eval_res = self.evaluator(target=str(target), thought=move_str)
                            
                        new_candidates.append({
                                "steps": current_path,
                                "available": actual_remaining,
                                "score": float(eval_res.assessment.score)
                            })
                    
                    except ValidationError as e:
                        print(f"Skipping invalid move (Pydantic): {e}")
                    except Exception as e:
                        print(f"Skipping invalid move (Logic): {e}")
                    
     
            # Selection step: Keep the top 4 best-scored paths (Beam Search)
            candidates = sorted(new_candidates, key=lambda x: x["score"], reverse=True)[:4]
            
        # After all depth levels are explored
        if not candidates:
            # Use 'path' and 'answer' so the print statement doesn't crash
            return dspy.Prediction(
                path="No valid math found", 
                answer="N/A", 
                score=0.0
            )
            

        # Use the Solver to wrap it all up
        best = candidates[0]
        if "24" in str(best["steps"]):
            # Use the solver to clean up the formatting of the successful path
            result = self.solver(problem=problem, reasoning_steps=best["steps"])
            return dspy.Prediction(answer=result.expression, path=best["steps"], score=10.0)
            
        result = self.solver(problem=problem, reasoning_steps=best["steps"])
        return dspy.Prediction(answer=result.expression, path=best["steps"], score=best["score"])
        

In [47]:
local_lm = dspy.LM('ollama_chat/'+MODEL, api_base='http://localhost:11434', api_key='')
local_lm.history = [] 
# Set it globally
#dspy.settings.configure(lm=local_lm)
dspy.settings.configure(lm=local_lm, cache=False) 


solver = TreeOfThoughts(depth=3)
problem_input = "1, 2, 4, 3"
#problem_input = "1, 5, 6, 3"
result = solver(problem=problem_input,target=24)

print("Best Path:", result.path)
print("Equation:", result.answer)
print("Score:", result.score)


depth is:+0
⚠️ Entire proposal block failed validation: 9 validation errors for list[MathStep]
0
  Value error, Field 'result' cannot be null. Propose a valid move. [type=value_error, input_value={'num1': 12, 'operator': ...ne, 'remaining': [None]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
1
  Value error, Field 'result' cannot be null. Propose a valid move. [type=value_error, input_value={'num1': 8, 'operator': '...ne, 'remaining': [None]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
2
  Value error, Field 'result' cannot be null. Propose a valid move. [type=value_error, input_value={'num1': 10, 'operator': ...ne, 'remaining': [None]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error
3
  Value error, Field 'result' cannot be null. Propose a valid move. [type=value_error, input_value={'num1': 12, 'operator': ...ne, 'remaining': 

In [48]:
num_calls = len(local_lm.history)
print(f"Total prompt calls made: {num_calls}")

Total prompt calls made: 2


In [49]:
dspy.inspect_history(n=42) # Set n high to catch everything





[2026-02-13T17:21:29.913518]

System message:

Your input fields are:
1. `target` (str): The number we are trying to reach
2. `thought` (str): The specific math move being scored
Your output fields are:
1. `reasoning` (str): 
2. `assessment` (Evaluation): Score and reasoning
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## target ## ]]
{target}

[[ ## thought ## ]]
{thought}

[[ ## reasoning ## ]]
{reasoning}

[[ ## assessment ## ]]
{assessment}        # note: the value you produce must adhere to the JSON schema: {"type": "object", "properties": {"reasoning": {"type": "string", "description": "Brief explanation of the score", "title": "Reasoning"}, "score": {"type": "number", "description": "A numeric score between 1 and 10", "title": "Score"}}, "required": ["score", "reasoning"], "title": "Evaluation"}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Evaluate if a mathematical move brings us cl